# Классификация текста

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

Мы будем решать задачу предсказания оценки, проставленной вину, базируясь на отзыве. Данные взяты из конкурса https://www.kaggle.com/zynicide/wine-reviews

In [ ]:
data = pd.read_csv('winemag.csv', index_col=0, na_filter=False)
data.head()

In [ ]:
data['points'].hist()

Бинаризуем таргет: будем предсказывать, превысит ли оценка 90.

In [ ]:
#Ваш код: бинаризуйте таргет
data.head()

Из всего многообразия признаков нас будет интересовать **description** -- признак содержащий описание вина и **points** -- присужденный балл

#### Задание 1
Выделите из данных X и y

In [ ]:
X = data['description']
y = data['>=90']

## Извлечение признаков из текстов

Перед тем как работать с текстом, его необходимо токенизировать - разбить на отдельные токены. В качестве токенов могут выступать слова, фразы, предложений и т.п. Токенизировать текст можно  помощью регулярных выражений или готовых токенизаторов. 

После токенизации нужно привести текст к нормальной форме. Речь идет о [стемминге и/или лемматизации](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html) - это схожие процессы, используемые для обработки словоформ.

Для работы лемматизации английского текста можно воспользоваться библиотекой nltk:

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

#### Задание 2
Токенизируйте тексты. Сейчас Х - это набор из строк-текстов. Разбейте каждый текст на слова (токенизируйте его), каждое из которых написано в нижнем регистре.

(Вам могут понадобиться функция word_tokenize из nltk.tokenize, а также методы строк isalpha и lower)

In [ ]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm_notebook

# [for x in tqdm_notebook(X)]

tokens = # ВАШ КОД ЗДЕСЬ

In [ ]:
tokens[0]

#### Задание 3
Теперь, когда для каждого текста у вас есть набор слов, встречающихся в нем, можно переходить к лемматизации.
Лемматизируйте все слова с помощью WordNetLemmatizer и объедините получившиеся слова в одну строку через пробел, запишите результат в X

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
#lemmatizer.lemmatize(<word>)

X = #ВАШ КОД

In [ ]:
X[0]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
y_train = y_train.values
y_test = y_test.values

### Bag of Words

Cоздаем вектор длиной в словарь, для каждого слова считаем количество вхождений в текст и подставляем это число на соответствующую позицию в векторе.

Построим модель BOW с помощью [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X_train)

vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d' % len(vocabulary))

description_count = vectorizer.transform(X_train)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

Видно, что большая часть из топ-10 слов является не информативными - стоп-словами. Что бы они не участвовали в представлении, в конструктор CountVectorizer в качестве параметра можно передать список стоп-слов:

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('english'))
vectorizer = CountVectorizer(stop_words=stop_words)
vectorizer.fit(X_train)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d' % len(vocabulary))

description_count = vectorizer.transform(X_train)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

Чтобы сжать векторное представление, можно "отбросить" редкие слова:

In [ ]:
vectorizer = CountVectorizer(stop_words=stop_words, min_df=10).fit(X)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

X_train_bow = vectorizer.transform(X_train)
X_test_bow = vectorizer.transform(X_test)

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nb_model = 
#ВАШ КОД: обучите модель наивного Байеса
y_pred = #ВАШ КОД: предскажите вероятности с помощью модели

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
plt.plot(*roc_curve(y_test, y_pred)[:2], label='test roc_auc, BOW model')

plt.plot([0, 1], [0, 1], '--', color='black',)
plt.grid()
plt.legend()

print(roc_auc_score(y_test, y_pred))

In [ ]:
def plot_models(names, models, X_train, X_test, y_train, y_test):
    """
    gets lists of names, models, lists of train and test samples, train and test answers
    plots ROC curves
    """
    
    for name, model, X in zip(names, models, X_train):
        proba = model.predict_proba(X)[:, 1]
        auc = roc_auc_score(y_train, proba)
        plt.plot(*roc_curve(y_train, proba)[:2], label='train %s AUC=%.4f' % (name, auc))
    
    for name, model, X in zip(names, models, X_test):
        proba = model.predict_proba(X)[:, 1]
        auc = roc_auc_score(y_test, proba)
        plt.plot(*roc_curve(y_test, proba)[:2], label='test %s AUC=%.4f' % (name, auc))
        
        
    plt.plot([0, 1], [0, 1], '--', color='black',)
    plt.legend(fontsize='large')
    plt.grid()
    plt.show()

In [ ]:
plot_models(['nb'], [nb_model], [X_train_bow], [X_test_bow], y_train, y_test)

### Логистическая регрессия: сравним качество

In [ ]:
#ВАШ КОД: обучите модель логистической регрессии

In [ ]:
plot_models(['nb', 'logistic regression'], [nb_model, logistic_model], 
            [X_train_bow] * 2, [X_test_bow] * 2, 
            y_train, y_test)

### Добавление tf-idf фичей

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf_idf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=10).fit(X)
vocabulary = tf_idf_vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

X_train_tf_idf = tf_idf_vectorizer.transform(X_train)
X_test_tf_idf = tf_idf_vectorizer.transform(X_test)

In [ ]:
tf_idf_logistic_model = LogisticRegression()

tf_idf_logistic_model.fit(X_train_tf_idf, y_train)

In [ ]:
plot_models(['bow', 'tf-idf'], [logistic_model, tf_idf_logistic_model], 
            [X_train_bow, X_train_tf_idf], [X_test_bow, X_test_tf_idf] * 2, 
            y_train, y_test)

### Кодирование предложений эмбеддингами слов

In [ ]:
import gensim.downloader 
embeddings = gensim.downloader.load("glove-wiki-gigaword-100")

In [ ]:
def vectorize_sum(comment):
    """
    implement a function that converts preprocessed comment to a sum of token vectors
    """
    embedding_dim = embeddings.wv.vectors.shape[1]
    
    comment_vectors = np.stack([embeddings.wv.word_vec(word) if word in embeddings.wv.vocab 
                                else np.zeros(embeddings.wv.vectors.shape[1])
                       for word in comment.split()])
    
    features = np.sum(comment_vectors, axis=0)
    return features

In [ ]:
X_train_wv = np.stack([vectorize_sum(text) for text in X_train])
X_test_wv = np.stack([vectorize_sum(text) for text in X_test])

In [ ]:
wv_model = LogisticRegression().fit(X_train_wv, y_train)

In [ ]:
plot_models(['bow', 'vw'], [logistic_model, wv_model], 
            [X_train_bow, X_train_wv], [X_test_bow, X_test_wv] * 2, 
            y_train, y_test)